In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15834 entries, 0 to 15833
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   img_path          15834 non-null  object
 1   upscale_img_path  15834 non-null  object
 2   label             15834 non-null  object
dtypes: object(3)
memory usage: 371.2+ KB


In [7]:
df['label'].value_counts()
len(df['label'].unique())
# get trained model and show wrong instances


25

In [8]:
CFG = {
    # 'IMG_SIZE': 224,
    'EPOCHS': 50,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 96,
    'SEED': 42,
    'SPLIT_PROPORTION': 0.2,
    'BASE_ARCH': 'mobilenet_v3_small'
}

# convert dict to string
CFG_STR = '_'.join([f'{k}_{v}' for k, v in CFG.items()])
CFG_STR

'EPOCHS_50_LEARNING_RATE_0.0003_BATCH_SIZE_96_SEED_42_SPLIT_PROPORTION_0.2_BASE_ARCH_mobilenet_v3_small'

In [9]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models
backbone = getattr(models, 'swin_v2_s')(pretrained=True) # models.mobilenet_v3_small(pretrained=True)

In [25]:

CFG = {'BASE_ARCH': 'resnet50', 'IMG_SIZE': 224}

# Transformations
train_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()])

test_transform = A.Compose([
    A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2()])


In [29]:
test_transform
type(preprocess), preprocess

(torchvision.transforms._presets.ImageClassification,
 ImageClassification(
     crop_size=[224]
     resize_size=[232]
     mean=[0.485, 0.456, 0.406]
     std=[0.229, 0.224, 0.225]
     interpolation=InterpolationMode.BILINEAR
 ))

In [23]:

# torchvision.models transform
if CFG['BASE_ARCH'] == 'resnet50':
    weights = models.ResNet50_Weights.DEFAULT
    preprocess = weights.transforms()
elif CFG['BASE_ARCH'] == 'efficientnet_v2_l':
    transforms = models.EfficientNet_V2_L_Weights.IMAGENET1K_V1.transforms
elif CFG['BASE_ARCH'] == 'efficientnet_b7':
    transforms = models.EfficientNet_B7_Weights.IMAGENET1K_V1.transforms
elif CFG['BASE_ARCH'] == 'swin_v2_s':
    transforms = models.Swin_V2_S_Weights.IMAGENET1K_V1.transforms

# import pdb; pdb.set_trace()

train_transform = preprocess
test_transform = preprocess

# add totensor to transform
train_transform.transforms.append(ToTensorV2())
train_transform

AttributeError: 'ImageClassification' object has no attribute 'transforms'

torchvision.transforms._presets.ImageClassification

In [13]:
import torchvision
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
weights.transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

In [1]:
import os
from PIL import Image
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageRegression, TrainingArguments, Trainer

# Load your dataset
dataset = load_dataset(
    "imagefolder",
    data_dir="path/to/your/bird/image/dataset",
    split="train"
)

# Define the pre-trained model and processor
model_name = "nvidia/swinir"
processor = AutoImageProcessor.from_pretrained(model_name)

# Define the transformation function
def transform(example_batch):
    images = [Image.fromarray(img.convert("RGB")) for img in example_batch["image"]]
    pixel_values = processor(images, return_tensors="pt").pixel_values
    return {"pixel_values": pixel_values}

# Apply the transformation to the dataset
transformed_dataset = dataset.with_transform(transform)

# Set up the model for fine-tuning
model = AutoModelForImageRegression.from_pretrained(model_name)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="path/to/output/directory",
    per_device_train_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    learning_rate=2e-5,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="psnr",
    push_to_hub=False,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=transformed_dataset,
)

# Fine-tune the model
trainer.train()

ImportError: cannot import name 'AutoModelForImageRegression' from 'transformers' (/home/st/anaconda3/envs/bird/lib/python3.11/site-packages/transformers/__init__.py)

In [2]:
import torch
import numpy as np
from PIL import Image
import requests

from transformers import AutoImageProcessor, Swin2SRForImageSuperResolution

processor = AutoImageProcessor.from_pretrained("caidas/swin2SR-classical-sr-x2-64")
model = Swin2SRForImageSuperResolution.from_pretrained("caidas/swin2SR-classical-sr-x2-64")

url = "https://huggingface.co/spaces/jjourney1125/swin2sr/resolve/main/samples/butterfly.jpg"
image = Image.open(requests.get(url, stream=True).raw)
# prepare image for the model
inputs = processor(image, return_tensors="pt")

# forward pass
with torch.no_grad():
    outputs = model(**inputs)

output = outputs.reconstruction.data.squeeze().float().cpu().clamp_(0, 1).numpy()
output = np.moveaxis(output, source=0, destination=-1)
output = (output * 255.0).round().astype(np.uint8)  # float32 to uint8
# you can visualize `output` with `Image.fromarray`

preprocessor_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/48.5M [00:00<?, ?B/s]

In [2]:
# read image from test and save it to upscale_test
import os
import torch
from PIL import Image
from RealESRGAN import RealESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=4)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)

indir = './test'
outdir = './upscale_test/real_esrgan_synthetic'
os.makedirs(outdir, exist_ok=True)

for root, dirs, files in os.walk(indir):
    for file in files:
        if file.endswith('.jpg'):
            path_to_image = os.path.join(root, file)
            image = Image.open(path_to_image).convert('RGB')
            sr_image = model.predict(image)
            sr_image.save(os.path.join(outdir, file))
            # break
# path_to_image = '/home/st/dacon/bird_cls/train/TRAIN_00001.jpg'
# image = Image.open(path_to_image).convert('RGB')

# sr_image = model.predict(image)

# sr_image.save('./sample_sr_image.png')


In [ ]:
# infer using trained model
from pl_baseline import BaseModel

# Instantiate the model
model = BaseModel()

# load ckpt
ckpt_path = '/home/st/dacon/bird_cls/lightning_logs/baseline_model/version_3/checkpoints/epoch=49-step=17350.ckpt'
model = BaseModel.load_from_checkpoint(ckpt_path)
model.eval()

preds = trainer.predict(model, test_loader)
preds = torch.cat([output for output in preds])
# import pdb; pdb.set_trace()
preds = le.inverse_transform(preds.argmax(1).cpu().numpy())  # Reshape preds to be 1-dimensional

# Submission
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = preds
submit.to_csv(f'./submit_{experiment_name}.csv', index=False)



# infer using image upscaler and trained classifier

In [ ]:
# https://github.com/wyhuai/ddnm 
python main.py --ni --config imagenet_256_cc.yml --path_y imagenet --eta 0.85 --deg "sr_bicubic" --deg_scale 4.0 --sigma_y 0 -i demo
python main.py --ni --simplified --config celeba_hq.yml --path_y solvay --eta 0.85 --deg "sr_averagepooling" --deg_scale 4.0 --sigma_y 0.1 -i demo

# Use upscale model

In [ ]:
def load_upscaler():
    from pl_resnet_sr import SuperResolutionLitModule
    ckpt_path = './lightning_logs/sr_resnet_mini_EPOCHS_100_LEARNING_RATE_0.0003_BATCH_SIZE_96_SEED_42/version_0/checkpoints/epoch=99-step=40000.ckpt'
    upscaler = SuperResolutionLitModule()
    upscaler.load_from_checkpoint(ckpt_path, map_location='cpu')
    upscaler.eval()
    return upscaler
upscaler = load_upscaler()

In [19]:
from PIL import Image
import torchvision
image = Image.open('./train/TRAIN_00002.jpg')
image.size
# to tensor
image = torchvision.transforms.ToTensor()(image)
# image.shape
upscaler(image.unsqueeze(0))

torch.Size([1, 3, 256, 256])

In [ ]:
# predict using pre-trained upsclaer and classifier model
from pl_resnet_sr import SuperResolutionLitModule
from pl_baseline import BaseModel
from PIL import Image
import torchvision
import torch

# Load models
upscaler = load_upscaler()
classifier = BaseModel.load_from_checkpoint('./lightning_logs/baseline_model/version_3/checkpoints/epoch=49-step=17350.ckpt')
classifier.eval()

# Load image
image = Image.open('./train/TRAIN_00002.jpg')
image = torchvision.transforms.ToTensor()(image)


In [20]:
# fixing #

# %% 
import random
import pandas as pd
import numpy as np
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm import tqdm

import pytorch_lightning as pl
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

# PyTorch Lightning imports
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

def load_upscaler():
    from pl_resnet_sr import SuperResolutionLitModule
    ckpt_path = './lightning_logs/sr_resnet_mini_EPOCHS_100_LEARNING_RATE_0.0003_BATCH_SIZE_96_SEED_42/version_0/checkpoints/epoch=99-step=40000.ckpt'
    upscaler = SuperResolutionLitModule()
    upscaler.load_from_checkpoint(ckpt_path, map_location='cpu')
    upscaler.eval()
    return upscaler

# device = torch.device('cuda:1') if torch.cuda.is_available() else torch.device('cpu')

# Hyperparameter Setting
CFG = {
    # 'IMG_SIZE': 224,
    'EPOCHS': 60,
    'LEARNING_RATE': 3e-4,
    'BATCH_SIZE': 24, # 93
    'SEED': 42,
    'SPLIT_PROPORTION': 0.2,
    'BASE_ARCH':'swin_v2_s' # efficientnet_v2_l efficientnet_b7 resnet50 swin_v2_s 
}

# Fixed RandomSeed
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

# Train & Validation Split
df = pd.read_csv('./train.csv')
train, val, _, _ = train_test_split(df, df['label'], test_size=CFG['SPLIT_PROPORTION'], stratify=df['label'], random_state=CFG['SEED'])

# Label-Encoding
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

# CustomDataset
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None, preprocess=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        self.preprocess = preprocess
        # 
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        image = cv2.imread(img_path)
        # import pdb; pdb.set_trace()
        if self.transforms is not None:
            image = self.transforms(image=image)['image']

        if self.preprocess is not None:
            
            image = self.preprocess(image)

        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image

    def __len__(self):
        return len(self.img_path_list)

preprocess = None
# torchvision.models transform
if CFG['BASE_ARCH'] == 'resnet50':
    weights = models.ResNet50_Weights.DEFAULT
    preprocess = weights.transforms()
elif CFG['BASE_ARCH'] == 'efficientnet_v2_l':
    preprocess = models.EfficientNet_V2_L_Weights.IMAGENET1K_V1.transforms()
elif CFG['BASE_ARCH'] == 'efficientnet_b7':
    preprocess = models.EfficientNet_B7_Weights.IMAGENET1K_V1.transforms()
elif CFG['BASE_ARCH'] == 'swin_v2_s':
    preprocess = models.Swin_V2_S_Weights.IMAGENET1K_V1.transforms()

upscaler = load_upscaler()
# Transformations
train_transform = A.Compose([
    # A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2(),
    # upscaler
    ])

test_transform = A.Compose([
    # A.Resize(CFG['IMG_SIZE'], CFG['IMG_SIZE']),
    # A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
    ToTensorV2(),
    # upscaler
    ])

train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform, preprocess=preprocess)
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform, preprocess=preprocess)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

# Test loader
test = pd.read_csv('./test.csv')
# test = pd.read_csv('./test_upscale.csv')
test_dataset = CustomDataset(test['img_path'].values, None, test_transform, preprocess=preprocess)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)


In [24]:
preprocess

ImageClassification(
    crop_size=[256]
    resize_size=[260]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)